# **GOR-III Algorithm Implementation**

In [ ]:
# Importing the necessary libraries to work with.
from math import *
import pandas as pd,numpy as np

# Reading the provided datasets using pandas library.
cath_File = pd.read_csv("/content/cath_info.txt",delimiter = "\t",header=None, names=["PDB_Code", "PDB_Chain_Code", "Prediction"])

dssp_File = pd.read_csv("/content/dssp_info.txt", delimiter = "\t",header=None, names=["PDB_Code", "PDB_Chain_Code", "SEQ_Pos","Residue_Name","Prediction"])

stride_File = pd.read_csv("/content/stride_info.txt",delimiter = "\t",header=None, names=["PDB_Code", "PDB_Chain_Code", "SEQ_Pos","Residue_Name","Prediction"])

# Defining the Amino Acid and the structure type as dictionaries.
conformation = {'H': 0, 'E': 1, 'C': 2}

amino_acids = {'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C', 'GLU': 'E', 'GLN': 'Q', 'GLY': 'G', 'HIS': 'H','ILE': 'I', 'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P', 'SER': 'S', 'THR': 'T','TRP': 'W', 'TYR': 'Y', 'VAL': 'V'}

amino_acids_count = {'ALA': 0, 'ARG': 0, 'ASN': 0, 'ASP': 0, 'CYS': 0, 'GLU': 0, 'GLN': 0, 'GLY': 0, 'HIS': 0,
           'ILE': 0, 'LEU': 0, 'LYS': 0, 'MET': 0, 'PHE': 0, 'PRO': 0, 'SER': 0, 'THR': 0,
           'TRP': 0, 'TYR': 0, 'VAL': 0}

#  The funtion replaces Other,Coil,Beta and Helix in the dataset which we give to a single letter as C,E and H respectively.
def changeNomenclature(dataset):
  dataset.replace('Other','C',inplace=True)
  dataset.replace('Coil','C',inplace=True)
  dataset.replace('Beta','E',inplace=True)
  dataset.replace('Helix','H',inplace=True)

# Single for loop which changes for both the datasets one after the other.
datasetList = [dssp_File,stride_File]
for i in datasetList:
  changeNomenclature(i)

# Printing the first few values of the dataset to check wether the changes are made correctly.
print("Stide Dataset")
print(stride_File.head())
print("\nDSSP Dataset")
print(dssp_File.head())
print("\nCath Dataset")
print(cath_File.head())

# Calculate the number of times a particular residue appears in a given dataset. Returns the count and updates the value in a dictionary. 
def residueCount(dataset):
    res_list = []
    for resname in dataset:
        if resname in amino_acids.keys():
            counter = amino_acids_count.get(resname)
            counter += 1
            amino_acids_count.update({resname: counter})
            res_list.append(resname)
    print(amino_acids_count)
    return res_list

# Calculate the number of times a particular conformation(structure) that is either Helix, Beta and Coil appears in a given dataset. Returns the count and updates the value in a dictionary.
def calcualate_fs(dataset):
  helix_count,beta_count,coil_count = 0,0,0
  for protstructure in dataset:
    if protstructure[4] == 'H':helix_count += 1
    elif protstructure[4] == 'E':beta_count += 1
    elif protstructure[4] == 'C':coil_count += 1
  fs = {}
  fs['Helix'] = helix_count
  fs['Beta'] = beta_count
  fs['Coil'] = coil_count
  return fs

# Printing the count of residues and the count of Helix,Beta and Coil Structure in the DSSP and the Stride dataset respectively.
print('\nThe count of residues in the STRIDE dataset is as follows: ')
stride_File_count = residueCount(list(stride_File['Residue_Name']))

print('\nThe count of residues in the DSSP dataset is as follows: ')
dssp_File_count = residueCount(list(dssp_File['Residue_Name']))

print('\nCount for STRIDE data set')
print(calcualate_fs(stride_File.values.tolist()))

print('\nCount for DSSP data set')
print(calcualate_fs(dssp_File.values.tolist()))

# Change the amino acid names into a single alphabet respectively from the amino_acids dictionary mentioned above.
dssp_File.Residue_Name.replace(amino_acids.keys(),amino_acids.values(),inplace=True)
stride_File.Residue_Name.replace(amino_acids.keys(),amino_acids.values(),inplace=True)

# Converting the complete dataset into a list . This is done in order to reduce the computing time by having a faster iteration through the list when compared to using iterrows() in pandas.
sList = stride_File.values.tolist()
dList = dssp_File.values.tolist()
pFamily = list(set(dssp_File.PDB_Code.to_list()))

# This function is used to calculate the Self Information between the Residues and the Conformation.It returns a dictionary of of the Helix,Beta and Coil info of the acids respectively.
def _calc_self_info(dataset,pdb):
   
    selfInformation_val = {}
    fs = calcualate_fs(dataset)
    H_fs = fs['Helix']
    E_fs = fs['Beta']
    C_fs = fs['Coil']
    H_fsr = 0
    E_fsr = 0
    C_fsr = 0

    for row in dataset:
        if row[0] == pdb:
            current = row[3]
            if row[3] in list(amino_acids.values()):
                H_fsr,E_fsr,C_fsr = 0,0,0
            for other in dataset:
                if other[0] != pdb:
                    if other[3] == current:
                        if other[4] == 'H':H_fsr += 1
                        if other[4] == 'E':E_fsr += 1
                        if other[4] == 'C':C_fsr += 1
            helix_info = log(H_fsr / ((H_fsr + E_fsr + C_fsr) - H_fsr)) + log((E_fs + C_fs) / H_fs)
            sheet_info = log(E_fsr / ((H_fsr + E_fsr + C_fsr) - E_fsr)) + log((H_fs + C_fs) / E_fs)
            coil_info = log(C_fsr / ((H_fsr + E_fsr + C_fsr) - C_fsr)) + log((H_fs + E_fs) / C_fs)
            selfInformation_val[current] = (helix_info, sheet_info, coil_info)

    return selfInformation_val
    

# This function calculates the acid pair information for the possible 16 neighbours and returns the dictionary with the acid name and the Helix, Beta and Coil Info along with the position of the acid.
def pairInformation(dataset,pdb):
    pair_Information = {} 
    fs = calcualate_fs(dataset)
    H_fs = fs['Helix']
    E_fs = fs['Beta']
    C_fs = fs['Coil']

    # To loop over 20 natural amino acids.
    for curr_acid in list(amino_acids.values()): 
        amino_acid_dict = {}
        # To loop over 16 neighbors.
        for m in [-8, -7, -6, -5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7, 8]:
            neighbour_pos = {}
            # To loop over 20 natural amino acids.
            for pair_acid in list(amino_acids.values()):
                H_fsr,E_fsr,C_fsr = 0,0,0
                # To loop over all the rest of the residues
                for i in range(len(dataset)-8):
                    if pair_acid == dataset[i+m][3]:
                         if dataset[i+m][0] == dataset[i][0]:
                              if dataset[i][0] != pdb:
                                   if dataset[i][3] == curr_acid:
                                       if dataset[i][4] == 'H':H_fsr += 1
                                       if dataset[i][4] == 'E':E_fsr += 1
                                       if dataset[i][4] == 'C':C_fsr += 1

                if ((H_fsr + E_fsr + C_fsr) - H_fsr) != 0:
                    helix_log_arg = H_fsr / ((H_fsr + E_fsr + C_fsr) - H_fsr)
                else:helix_log_arg = 1

                if ((H_fsr + E_fsr + C_fsr) - E_fsr) != 0:
                    sheet_log_arg = E_fsr / ((H_fsr + E_fsr + C_fsr) - E_fsr)
                else: sheet_log_arg = 1

                if ((H_fsr + E_fsr + C_fsr) - C_fsr) != 0:
                    coil_log_arg = C_fsr / ((H_fsr + E_fsr + C_fsr) - C_fsr)
                else: coil_log_arg = 1

                helix_info = log(helix_log_arg if helix_log_arg > 0 else 1) + log((E_fs + C_fs) / H_fs)
                sheet_info = log(sheet_log_arg if sheet_log_arg > 0 else 1) + log((H_fs + C_fs) / E_fs)
                coil_info = log(coil_log_arg if coil_log_arg > 0 else 1) + log((H_fs + E_fs) / C_fs)

                neighbour_pos[pair_acid] = (helix_info, sheet_info, coil_info)
            
            amino_acid_dict[m] = neighbour_pos
        pair_Information[curr_acid] = amino_acid_dict
    
    return pair_Information

# This function is used for the secondary structure prediction of the protein in the form of a list.
def sec_struc_prediction(dataset,PDB_Code):

    selfInfo = _calc_self_info(dataset,PDB_Code)
    pairInfo = pairInformation(dataset,PDB_Code)

    prot_list = [entry for entry in dList]
    protein = ''
    real_structure = ''
    prediction = [PDB_Code]
    predicted_structure = ''
    
    # To get the real protein secondary structure in the form of a string.
    for acid in prot_list:
        protein += acid[3].replace("  ","")
        
        if acid[4] == 'H':
            real_structure += 'H'
        elif acid[4] == 'E': 
            real_structure += 'E'
        elif acid[4] == 'C' : 
            real_structure += 'C'
    
  
    # To get the protein secondary stucture prediction in the form of a string
    for i in range(len(protein)):
        if protein[i] in selfInfo:
            helix = [selfInfo[protein[i]][0],'H']
            sheet = [selfInfo[protein[i]][1],'E']
            coil = [selfInfo[protein[i]][2],'C']
        
        for m in [-8, -7, -6, -5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7, 8]:
            if i ==0 : continue
            if i+m > 0 and i+m < len(protein):
                if protein[i] in pairInfo and protein[i+m] in pairInfo:
                    helix[0] += pairInfo[protein[i]][m][protein[i+m]][0]
                    sheet[0] += pairInfo[protein[i]][m][protein[i+m]][1]
                    coil[0] += pairInfo[protein[i]][m][protein[i+m]][2]


        if max(helix[0], sheet[0], coil[0]) == helix[0]: 
            predicted_structure += helix[1]
        if max(helix[0], sheet[0], coil[0]) == sheet[0]:
             predicted_structure += sheet[1]
        if max(helix[0], sheet[0], coil[0]) == coil[0]: 
            predicted_structure += coil[1]
        
    prediction.append(predicted_structure)
    
    # To Calculate the Q3 Score based on the predictions.
    predicted_residues = 0
    for i in range(len(real_structure)):
        if real_structure[i] == predicted_structure[i]:
            predicted_residues += 1
    
    q3 = (predicted_residues / len(real_structure))*100
    prediction.append(q3)


   # Calculation of the MCC score based on the predicted structure and the real structure.
    # aa = []
    # for i,j in zip(structure_prediction,structure_real):
    #   c = 0 
    #   table = np.zeros(shape=(3,4),dtype=int).tolist()
    #   for p,f in zip(i,j):
    #     if p == f[-1]:
    #       c+=1
    #     for x in range(3):
    #       res = list(stru_dict.keys())[x]
    #       if f[-1]== res:
    #         if f[-1]== i:
    #           table[x][0]+= 1
    #         else:
    #           table[x][2]+=1

    #       else:
    #         if i == res:
    #           table[x][1]+=1
    #         else :
    #           table[x][3]+=1

    # mcc = []
    # for t in table:
    #   TP = t[0]
    #   FP = t[1]
    #   FN = t[2]
    #   TN = t[3]

    #   try:
    #     m = (TP*TN-FP*FN)/sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    #   except ZeroDivisionError :
    #     m = (TP*TN-FP*FN)

    #   mcc.append(m)

    #   prediction.append(mcc)



    return prediction


# Predicting the Secondary Structure for 10 Proteins (pFamily in our case) due to RAM consumptions
result_stride,result_dssp = [],[]

for prot in pFamily[:5]:
    result_dssp.append(sec_struc_prediction(dList,prot))

for i in pFamily[:5]:
  result_stride.append(sec_struc_prediction(dList,prot))

Stide Dataset
  PDB_Code PDB_Chain_Code  SEQ_Pos Residue_Name Prediction
0     1w0n              A       12          ILE          C
1     1w0n              A       13          THR          E
2     1w0n              A       14          LYS          E
3     1w0n              A       15          VAL          E
4     1w0n              A       16          GLU          E

DSSP Dataset
  PDB_Code PDB_Chain_Code  SEQ_Pos Residue_Name Prediction
0     1w0n              A       12          ILE          C
1     1w0n              A       13          THR          E
2     1w0n              A       14          LYS          E
3     1w0n              A       15          VAL          E
4     1w0n              A       16          GLU          E

Cath Dataset
  PDB_Code PDB_Chain_Code  Prediction
0     1w0n              A        Beta
1     2gpi              A  Alpha/beta
2     1vbw              A  Alpha/beta
3     2odk              A  Alpha/beta
4     2zxy              A       Alpha

The count of residues

In [44]:
# This function returns the cumulative Q3 values for the given dataset Stride and DSSP respectively.
def cum_q3(dataset):
  q3Value = [i[-1] for i in dataset]
  return sum(q3Value) / len(dataset)

cum_stride = cum_q3(result_stride)
cum_dssp = cum_q3(result_dssp)

print(f'The cumulative Q3 value for Stride dataset is {cum_stride} (Reason being 48% is i considered the 10 proteins)') 
print(f'The cumulative Q3 value for Dssp dataset is {cum_dssp} (Reason being 48% is i considered the 10 proteins)')

The cumulative Q3 value for Stride dataset is 48.06628286478688 (Reason being 48% is i considered the 10 proteins)
The cumulative Q3 value for Dssp dataset is 48.70431847151602 (Reason being 48% is i considered the 10 proteins)


In [ ]:
# This performs the leave one out method where it pops the values based on the given condition from the dataset.
def LeaveOneOut(aList):
    for i,row in enumerate(aList):
        if row[3] == 'N' and row[4] == 'H':aList.pop(i)
        elif row[3] == 'N' and row[4] == 'E':aList.pop(i)
        elif row[3] == 'N' and row[4] == 'C':aList.pop(i)
    return aList
Stride_leave = LeaveOneOut(stride_File.values.tolist())
Dssp_leave = LeaveOneOut(dssp_File.values.tolist())

In [45]:
result_dssp[:3]

[['3fil',
  '...',
  48.836688097939515],
 ['1m1f',
  '...',
  49.014581669958396],
 ['1ryl',
  '...',
  48.50331274250256]]

In [46]:
result_stride[:3]

[['1m9z',
  '...',
  48.49770979535236],
 ['1m9z',
  '...',
  48.49770979535236],
 ['1m9z',
  '...',
  48.49770979535236]]

In [24]:
result_stride_leave,result_dssp_leave = [],[]

for prot in pFamily[:10]:
    result_dssp_leave.append(sec_struc_prediction(Dssp_leave,prot))

In [47]:
result_dssp_leave[:3]

[['3fil',
  '...',
  48.385650852348334],
 ['1m1f',
  '...',
  48.54253337255396],
 ['1ryl',
  '...',
  48.094297600537885]]

In [28]:
for i in pFamily[:10]:
  result_stride.append(sec_struc_prediction(Stride_leave,prot))

In [48]:
result_stride[:3]

[['1m9z',
  '...',
  48.49770979535236],
 ['1m9z',
  '...',
  48.49770979535236],
 ['1m9z',
  '...',
  48.49770979535236]]

### After reading the paper, "GOR Method for Predicting Protein Secondary Structure from Amino Acid Sequence" pubished by JEAN GARNIER, JEAN-FRANQOIS GIBRAT, and BARRY ROBSON I was able to understand that the GOR III algorithm had two parts:
  ### 1. Self Information Part : Frequency for a particular residue of a conformation in a given dataset.
  ### 2. Conditional (Residual Inforamtion) Part : This is also called as the pair information where we consider a single residue and compare it with the 16 other neighbours (8 before and after).
  ### This process is done for all the 20 Amino acids.

### Based on these information I have deviced the GOR III algorithm which predicts the secondary structure of the protein. 

### The Q3 score in my file will be around 48% for both GOR and Leave One out method because I have considered only for 10 Proteins.

### I tried Implementing the MCC Score for which the code can be seen commented above. 

### The single constraint which I faced was "*RAM Limitations in my computer*." 